In [44]:
import sys
import pandas as pd
sys.path.append('../src')
from restart import NoteCompose
from data import Data
from util import set_config, to_df, to_sheet, display_population, format_population, format_cells

In [53]:
import ipywidgets as widgets

state_box = widgets.Text(
    placeholder='Enter a US State',
    description='State:',
)

county_box = widgets.Text(
    placeholder='Enter a US County',
    description='County:',
)

button = widgets.Button(description="Update Model")

In [54]:
config = set_config('../src')
restart = NoteCompose(configdir='../src', population='oes', state='California')
model = restart.model
model.inventory.set_average_orders_per_period(model.demand.demand_by_popsum1_total_rp1n_tc)

slider = widgets.IntSlider(value=30, continuous_update=False)

def dashboard(state, county, backstop):
    if len(state) == 0:
        state = None
    if len(county) == 0:
        county = None
    restart.set_model(state=state, county=county)
    set_stock(backstop)
    
def display_stock(df):
    df = df.round()
    index_name = "Population"
    headers = ['Essential', 'Non-Essential']
    df.insert(loc=0, column=index_name, value=headers)
    sheet = to_sheet(df)
    format_cells(sheet)
    sheet.row_headers = False
    display(sheet)
    
def set_stock(backstop):
    backstop = [backstop]
    model.inventory.order(model.inventory.inv_by_popsum1_total_rp1n_tc)
    model.inventory.set_min_in_periods(backstop)
    display_stock(model.inventory.inv_by_popsum1_total_rp1n_tc.df)
    
out = widgets.interactive_output(dashboard, {'state': state_box, 'county': county_box, 'backstop': slider})

widgets.VBox([state_box, county_box, slider, out])